In [43]:
import argparse
from utils import prepare_save_dir
# from STELLAR import STELLAR
import numpy as np
import torch
import pandas as pd
# import anndata
# import scanpy as sc
import pickle
import sys
sys.path.append("../")
# from datasets import GraphDataset
from sklearn.metrics import f1_score, accuracy_score

In [44]:
parser = argparse.ArgumentParser(description='STELLAR')
parser.add_argument('--dataset', default='TonsilBE', help='dataset setting')
parser.add_argument('--seed', type=int, default=1, metavar='S', help='random seed (default: 1)')
parser.add_argument('--name', type=str, default='STELLAR')
parser.add_argument('--epochs', type=int, default=50)
parser.add_argument('--lr', type=float, default=1e-3) # learning rate
parser.add_argument('--wd', type=float, default=5e-2) # weight decay
parser.add_argument('--num-heads', type=int, default=13)
parser.add_argument('--num-seed-class', type=int, default=3)
parser.add_argument('--sample-rate', type=float, default=0.5) # downsample dataset by using 50% of cells
parser.add_argument('-b', '--batch-size', default=1, type=int,
                metavar='N', help='mini-batch size')
parser.add_argument('--distance_thres', default=50, type=int)# distance threshold for constructing the graph
parser.add_argument('--savedir', type=str, default='./') # output directory

parser.add_argument('--use-processed-graph', type=bool, default=False) # whether to use already preprocessed graph or construct the graph 

_StoreAction(option_strings=['--use-processed-graph'], dest='use_processed_graph', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, required=False, help=None, metavar=None)

In [45]:
args = parser.parse_args(args=[])
args.cuda = torch.cuda.is_available()
args.device = torch.device("cuda" if args.cuda else "cpu")

# Define User Params

In [209]:
dataset_path = './data/TonsilBE/BE_Tonsil_l3_dryad.csv'
raw_pred_path = './experiments/run/TonsilBE_STELLAR/TonsilBE_results__TonsilBE_epoch320_batch32_dist30_gat.npy'


In [210]:
df = pd.read_csv(dataset_path)
df.columns

Index(['Unnamed: 0', 'CHGA', 'PDL1', 'CD56', 'CK7', 'FoxP3', 'CD21', 'MUC1',
       'PD1', 'CD11b', 'CD4', 'CD31', 'CD25', 'CD15', 'CD20', 'Annexin A1',
       'aSMA', 'CD11c', 'Nestin', 'IDO', 'Cytokeratin', 'MUC5AC', 'Vimentin',
       'CD36', 'HLADR', 'BCL2', 'p63', 'CD3', 'CD45', 'CD8', 'CD57',
       'aDefensin5', 'CD68', 'CD34', 'CD38', 'Podoplanin', 'CD163', 'Bcatenin',
       'CD138', 'Arginase1', 'CD73', 'CD206', 'MUC6', 'COX2', 'MMP9', 'x', 'y',
       'sample_name', 'cell_type'],
      dtype='object')

In [211]:
df.head(2)

,Unnamed: 0,CHGA,PDL1,CD56,CK7,FoxP3,CD21,MUC1,PD1,CD11b,...,Arginase1,CD73,CD206,MUC6,COX2,MMP9,x,y,sample_name,cell_type
0,30,-0.015838,-0.262392,-0.328807,-0.030104,-0.169786,-0.195315,-0.14445,-0.314147,-0.306023,...,0.183429,-0.471492,0.368175,-0.137850,0.148542,-0.161118,394.0,3516.0,tonsil,Innate
1,36,-0.063117,-0.075990,1.852914,-0.030104,-0.169786,2.408519,-0.14445,1.413897,1.150271,...,3.152803,-0.460310,0.406059,0.247144,0.549028,-0.114117,5469.0,2463.0,tonsil,Innate


In [214]:

if "infected" in dataset_path.lower():
    print("infected")
    train_df = df.loc[df['region'] == 'healthy']
    test_df = df.loc[df['region'] == "infected"]

    train_y = train_df['cluster'].str.lower()
    test_y = test_df['cluster'].str.lower()
elif "cross" in dataset_path.lower():
    print("Cross")
    train_df = df.loc[df['region'] == 1]
    test_df = df.loc[df['region'] == 2]
    
    train_y = train_df['cluster'].str.lower()
    test_y = test_df['cluster'].str.lower()    
elif "tonsil" in dataset_path.lower():
    print("tonsil")
    train_df = df.loc[df['sample_name'] == 'tonsil']
    test_df = df.loc[df['sample_name'] == 'Barretts Esophagus']

    train_y = train_df['cell_type'].str.lower()
    test_y = test_df['cell_type'].str.lower()

cell_types = np.sort(list(set(test_y))).tolist()
cell_type_dict = {}
for i, cell_type in enumerate(cell_types):
    cell_type_dict[cell_type] = i

tonsil


In [215]:
cell_type_dict

{'endothelial': 0,
 'glandular_epi': 1,
 'innate': 2,
 'nerve': 3,
 'paneth': 4,
 'pdpn': 5,
 'plasma': 6,
 'secretory_epithelial': 7,
 'smoothmuscle': 8,
 'squamous_epithelial': 9,
 'stroma': 10,
 't': 11}

# Generate Class Label Mapping for Train and Test

In [216]:
if "infected" in dataset_path.lower():
    print("infected")
    train_df = df.loc[df['region'] == 'healthy']
    test_df = df.loc[df['region'] == "infected"]

    train_y = train_df['cluster'].str.lower()
    test_y = test_df['cluster'].str.lower()
elif "cross" in dataset_path.lower():
    print("Cross")
    train_df = df.loc[df['region'] == 1]
    test_df = df.loc[df['region'] == 2]
    
    train_y = train_df['cluster'].str.lower()
    test_y = test_df['cluster'].str.lower()    
elif "tonsil" in dataset_path.lower():
    print("tonsil")
    train_df = df.loc[df['sample_name'] == 'tonsil']
    test_df = df.loc[df['sample_name'] == 'Barretts Esophagus']

    train_y = train_df['cell_type'].str.lower()
    test_y = test_df['cell_type'].str.lower()
    



tonsil


In [217]:
cell_types_train = np.sort(list(set(train_y))).tolist()
class_train = [i for i in range(len(cell_types_train))]

cell_type_dict_train = {}
inverse_dict_train = {}

cell_types_test = np.sort(list(set(test_y))).tolist()
cell_type_dict_test = {}
inverse_dict_test = {}

for i, cell_type in enumerate(cell_types_train):
    cell_type_dict_train[cell_type] = i
    inverse_dict_train[i] = cell_type

for i, cell_type in enumerate(cell_types_test):
    cell_type_dict_test[cell_type] = i
    inverse_dict_test[i] = cell_type

In [218]:
cell_types_train

['b',
 'endothelial',
 'innate',
 'nerve',
 'pdpn',
 'plasma',
 'smoothmuscle',
 'squamous_epithelial',
 'stroma',
 't']

In [219]:
list(cell_types_test)

['endothelial',
 'glandular_epi',
 'innate',
 'nerve',
 'paneth',
 'pdpn',
 'plasma',
 'secretory_epithelial',
 'smoothmuscle',
 'squamous_epithelial',
 'stroma',
 't']

In [220]:
inverse_dict_train

{0: 'b',
 1: 'endothelial',
 2: 'innate',
 3: 'nerve',
 4: 'pdpn',
 5: 'plasma',
 6: 'smoothmuscle',
 7: 'squamous_epithelial',
 8: 'stroma',
 9: 't'}

In [221]:
inverse_dict_test

{0: 'endothelial',
 1: 'glandular_epi',
 2: 'innate',
 3: 'nerve',
 4: 'paneth',
 5: 'pdpn',
 6: 'plasma',
 7: 'secretory_epithelial',
 8: 'smoothmuscle',
 9: 'squamous_epithelial',
 10: 'stroma',
 11: 't'}

# Read in the raw preds


In [222]:
array_loaded = np.load(raw_pred_path)

In [223]:
pred_label = []
test_not_train_pred_ids = set()
for i, pred_class in enumerate(array_loaded):
    if pred_class not in class_train:
        pred_label.append(f'{pred_class}')
        test_not_train_pred_ids.add(f'{pred_class}')
        # pred_label.append('novel')
    
    else:
        known_cell_type = inverse_dict_train[pred_class]        
        pred_label.append(known_cell_type)

test_not_train_pred_ids = list(test_not_train_pred_ids)
test_not_train_pred_ids.sort()
test_not_train_pred_ids
# final = np.array([test_y,pred_label]).T
# print(len(final))
# final = np.delete(final,np.where(final == 'novel')[0],axis = 0)
# final = np.delete(final,np.where(~((final == 'glandular_epi') | (final == 'secretory_epithelial') | (final == 'paneth')))[0],axis = 0)
# row_idx, col_idx = np.where(~((final == 'glandular_epi') | (final == 'secretory_epithelial') | (final == 'paneth')))
# print(len(final))


['10', '11', '12']

In [224]:
test_y.sample(5)

188388           glandular_epi
180395    secretory_epithelial
197779           glandular_epi
182854           glandular_epi
198149           glandular_epi
Name: cell_type, dtype: object

In [225]:
pred_label[:5]

['innate', 'endothelial', 'endothelial', 'endothelial', 'endothelial']

In [226]:
# create df of labels and prds
results_df = pd.DataFrame({'label': test_y, 'pred': pred_label})
results_df = results_df.reset_index(drop=True)

In [227]:
results_df.head()


,label,pred
0,innate,innate
1,endothelial,endothelial
2,endothelial,endothelial
3,endothelial,endothelial
4,endothelial,endothelial


In [228]:
def preprocess_results_matrix(result_df, 
                              train_not_test_gt_labels,
                              test_not_train_gt_labels,
                              test_not_train_pred_ids):
    # novel_cell_types = ['glandular_epi', 'secretory_epithelial', 'paneth']
    # novel_cell_types = ['undefined']
    # drop novel cell types from the results df
        
    train_not_test_cond = ~result_df['label'].isin(train_not_test_gt_labels)
    test_not_train_cond1 = ~result_df['pred'].isin(test_not_train_gt_labels)
    test_not_train_cond2 = ~result_df['pred'].isin(test_not_train_pred_ids)
    
    filter_cond = train_not_test_cond & test_not_train_cond1 & test_not_train_cond2
    
    result_df_known = result_df.loc[filter_cond]

    return result_df_known



In [229]:
# get the list of novel cell types
cell_types_train = inverse_dict_train.values()
cell_types_test = inverse_dict_test.values()

train_not_test_gt_labels = set(cell_types_train) - set(cell_types_test) 
train_not_test_gt_labels = list(train_not_test_gt_labels)

test_not_train_gt_labels = set(cell_types_test) - set(cell_types_train)
test_not_train_gt_labels = list(test_not_train_gt_labels)


In [230]:
train_not_test_gt_labels

['b']

In [231]:
test_not_train_gt_labels

['glandular_epi', 'secretory_epithelial', 'paneth']

In [232]:
results_df_known  = preprocess_results_matrix(results_df, 
                                              train_not_test_gt_labels,
                                              test_not_train_gt_labels,
                                              test_not_train_pred_ids
                                              )

In [233]:
results_df_known['label'].value_counts()

label
smoothmuscle           9003
endothelial            6074
innate                 4245
stroma                 4180
nerve                  2037
t                      1388
plasma                 1157
pdpn                    909
squamous_epithelial     703
glandular_epi            74
paneth                    2
Name: count, dtype: int64

In [234]:
results_df_known['pred'].value_counts()

pred
b                      10886
endothelial             6900
innate                  4124
stroma                  3785
t                       1374
plasma                  1170
pdpn                     767
squamous_epithelial      649
smoothmuscle             108
nerve                      9
Name: count, dtype: int64

In [235]:
from sklearn.metrics import precision_recall_fscore_support, classification_report
# metrics = precision_recall_fscore_support(final[:,0],final[:,1],average = 'weighted')
classification = classification_report(results_df_known['label'], results_df_known['pred'])
print(classification)

/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

                     precision    recall  f1-score   support

                  b       0.00      0.00      0.00         0
        endothelial       0.85      0.97      0.90      6074
      glandular_epi       0.00      0.00      0.00        74
             innate       0.96      0.93      0.95      4245
              nerve       1.00      0.00      0.01      2037
             paneth       0.00      0.00      0.00         2
               pdpn       0.93      0.78      0.85       909
             plasma       0.95      0.96      0.95      1157
       smoothmuscle       0.85      0.01      0.02      9003
squamous_epithelial       0.95      0.88      0.91       703
             stroma       0.90      0.82      0.86      4180
                  t       0.96      0.95      0.96      1388

           accuracy                           0.57     29772
          macro avg       0.70      0.53      0.53     29772
       weighted avg       0.90      0.57      0.58     29772



/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [236]:
classification = classification_report(results_df['label'], results_df['pred'])
print(classification)

/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

                      precision    recall  f1-score   support

                  10       0.00      0.00      0.00         0
                  11       0.00      0.00      0.00         0
                  12       0.00      0.00      0.00         0
                   b       0.00      0.00      0.00         0
         endothelial       0.85      0.95      0.90      6181
       glandular_epi       0.00      0.00      0.00     14690
              innate       0.96      0.93      0.94      4282
               nerve       1.00      0.00      0.01      2047
              paneth       0.00      0.00      0.00       275
                pdpn       0.93      0.78      0.85       914
              plasma       0.95      0.94      0.95      1177
secretory_epithelial       0.00      0.00      0.00       658
        smoothmuscle       0.85      0.01      0.02      9023
 squamous_epithelial       0.95      0.57      0.71      1077
              stroma       0.90      0.81      0.85      4218
       

/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sukritrao/miniforge3/envs/rl_project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
